# Theravada Scripture Search POC

In this notebook, we'll create a proof-of-concept of search over the translated Theravadan texts from https://www.dhammatalks.org/suttas/.

## Setup

In [1]:

# Run once.

%load_ext autoreload
%autoreload 2

import os

# Move execution back one directory.
os.chdir('..')

In [ ]:
import re
from time import sleep

import pandas as pd
from IPython.display import HTML
from dotenv import load_dotenv
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pydantic import BaseModel
from smolagents import HfApiModel
from tqdm import tqdm

from scripture_search.retrievers.bm25 import ScriptureSearchTool

load_dotenv()
HTML(
    """
<style>
.output_scroll {
    max-height: 400px;
    overflow-y: auto;
}
</style>
"""
)

## Load data

In [3]:
dhamma_talks_suttas = pd.read_csv('data/dhamma_talks_suttas.csv', index_col=0)
dhamma_talks_suttas["id"] = dhamma_talks_suttas.index.to_list()
print(dhamma_talks_suttas.shape)
dhamma_talks_suttas.head()

(240, 9)


,collection,title,url_source,religion,subgroup,source,translation_source,text,id
0,AN,A Single Thing,https://www.dhammatalks.org/suttas/AN/AN1_21.html,Buddhism,Theravada,Dhamma Talks,Thanissaro Bhikkhu,"21. âI donât envision a single thing that,...",0
1,AN,A Pool of Water,https://www.dhammatalks.org/suttas/AN/AN1_45.html,Buddhism,Theravada,Dhamma Talks,Thanissaro Bhikkhu,45. âSuppose there were a pool of waterâsu...,1
2,AN,Soft,https://www.dhammatalks.org/suttas/AN/AN1_48.html,Buddhism,Theravada,Dhamma Talks,Thanissaro Bhikkhu,"âJust as, of all trees, the balsam is foremo...",2
3,AN,Quick to Reverse Itself,https://www.dhammatalks.org/suttas/AN/AN1_49.html,Buddhism,Theravada,Dhamma Talks,Thanissaro Bhikkhu,âI donât envision a single thing that is a...,3
4,AN,Luminous,https://www.dhammatalks.org/suttas/AN/AN1_50.html,Buddhism,Theravada,Dhamma Talks,Thanissaro Bhikkhu,"âLuminous, monks, is the mind.1 And it is de...",4


In [4]:

def decode_text(text):
    if not isinstance(text, str):
        return text
    try:
        # First try to decode as UTF-8
        return text.encode("latin1").decode("utf-8")
    except UnicodeDecodeError:
        try:
            # If that fails, try to decode as latin1 first
            return text.encode("latin1").decode("latin1")
        except:
            # If all else fails, return the original text
            return text

def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    print(df.shape)
    df = df.dropna(subset=["text"])
    print(df.shape)
    df = df.drop_duplicates(subset=["text"])
    print(df.shape)
    df["text"] = df["text"].apply(decode_text)
    return df


dhamma_talks_suttas = preprocess(dhamma_talks_suttas)
dhamma_talks_suttas.shape


(240, 9)
(239, 9)
(235, 9)


(235, 9)

## Create basic search

In [7]:
def row_to_doc(row: pd.Series) -> Document:
    return Document(
        page_content=row["text"], 
        metadata={
            "id": row["id"],
            "collection": row["collection"],
            "title": row["title"],
            "full_text": row["text"],
            "url_source": row["url_source"],
            "religion": row["religion"],
            "subgroup": row["subgroup"],
            "source": row["source"],
            "translation_source": row["translation_source"],
        }
    )

documents = dhamma_talks_suttas.apply(row_to_doc, axis=1).tolist()

# Split the documents into smaller chunks for more efficient search
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    add_start_index=True,
    strip_whitespace=True,
    separators=["\n\n", "\n", ".", " ", ""],
)
docs_processed = text_splitter.split_documents(documents)
print(len(docs_processed))

# # Create the retriever tool
scripture_search_tool = ScriptureSearchTool(docs_processed)

9158


In [8]:
search_results = scripture_search_tool.search("I struggle with motivation to meditate. What can I do?")
search_results

[Document(metadata={'id': 28, 'collection': 'MN', 'title': 'The Simile of the Cloth', 'full_text': "I have heard that on one occasion the Blessed One was staying near Sāvatthī in Jeta's Grove, Anāthapiṇḍika's monastery. There he addressed the monks: “Monks!”\n“Yes, lord,” the monks responded to him.\nThe Blessed One said, “Monks, suppose a piece of cloth were defiled and tenaciously stained, and a dyer were to dip it into some dye or other—whether blue or yellow or red or crimson. It would be poorly dyed, impurely dyed. Why is that? Because of the impurity of the cloth. In the same way, when the mind is defiled, a bad destination can be expected.\n“But suppose a piece of cloth were pure and clean, and a dyer were to dip it into some dye or other—whether blue or yellow or red or crimson. It would be well dyed, purely dyed. Why is that? Because of the purity of the cloth. In the same way, when the mind is undefiled, a good destination can be expected.\n“And which are the defilements of t

In [9]:
search_results = scripture_search_tool.search("How can I learn how to be mindful?")
search_results

[Document(metadata={'id': 77, 'collection': 'MN', 'title': 'At KÄ«á¹\xadÄ\x81giri', 'full_text': 'I have heard that on one occasion the Blessed One was wandering on a tour of Kāsi with a large Saṅgha of monks. There he addressed the monks: “I abstain from the nighttime meal.1 As I am abstaining from the nighttime meal, I sense next-to-no illness, next-to-no affliction, lightness, strength, & comfortable abiding. Come, now. You, too, abstain from the nighttime meal. As you are abstaining from the nighttime meal, you, too, will sense next-to-no illness, next-to-no affliction, lightness, strength, & comfortable abiding.”\n“As you say, lord,” the monks responded to him.\nThen, as he was wandering by stages in Kāsi, the Blessed One eventually arrived at a Kāsi town called Kīṭāgiri. And there he stayed in the Kāsi town, Kīṭāgiri.\nNow at that time the monks led by Assaji & Punabbasu2 were residing in Kīṭāgiri. Then a large number of monks went to them and, on arrival, said to them, “The Bles

These are solid but let's go a step further and use an Agent to curate these based on the query.

## Agentic Librarian approach
With this approach, the agent will only directly output text from the suttas and will not try to interpret the text at all.

In [10]:
system_prompt = """You are a helpful librarian specializing in Buddhist texts and teachings.
A user will provide a query, and you will be provided with a sutta that may or may not be 
relevant to the query. Provide a brief statement with the following format:

Does this sutta fully address the user query, why or why not: {explanation} 
"""
scoring_prompt = """Given the user query, sutta, and the statement on the appropriateness of the sutta
to the user query, output a score from 0 and 10 based on how well the sutta addresses the user's query.
Output the score number and nothing else.
"""

In [11]:

def create_message(role: str, text: str) -> dict[str, str]:
    return {"role": role, "content": [{"type": "text", "text": text}]}

def get_sutta_str(doc: Document) -> str:
    return f"""
    Sutta Title: {doc.metadata["title"]} \n
    Sutta Full text: {doc.metadata["full_text"]}
    """

def query_for_score(base_messages: list[dict[str, str]], doc: Document) -> int:
    print("Calling LLM with sutta and user query...")
    messages = list(base_messages)
    messages.append(create_message("assistant", get_sutta_str(doc)))
    response = librarian_model(messages)

    print("Received response: ", response.content)
    messages.append(create_message("assistant", scoring_prompt))
    print(messages)
    # raise Exception
    response = librarian_model(messages)

    print("Received score response: ", response.content)
    print("Trying to parse score...")
    try:
        score = int(re.search(r'\d+', response.content).group())
    except:
        score = 0
    print("Parsed score: ", score)
    
    return score

In [19]:

from scripture_search.models.open_ai import OpenAIModel

# model_id = "meta-llama/Llama-3.2-3B-Instruct"
# model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
# librarian_model = HfApiModel(
#     model_id=model_id,
#     token=os.getenv("HF_TOKEN"),
#     # max_tokens=5000,
# )
librarian_model = OpenAIModel()

In [20]:

class Result(BaseModel):
    doc: Document
    score: int

def get_results(user_query: str, k: int = 10):
    similar_docs = scripture_search_tool.search(user_query, k=k)
    scores = []
    base_messages = [
        create_message("system", system_prompt),
        create_message("user", user_query),
    ]
    for doc in tqdm(similar_docs):
        scores.append(query_for_score(base_messages, doc))
        sleep(1)
    results = [Result(doc=doc, score=score) for doc, score in zip(similar_docs, scores)]
    results = sorted(results, key=lambda x: x.score, reverse=True)
    return results

In [21]:
user_query = "I struggle with motivation to meditate. What can I do?"
K_RESULTS = 10
results = get_results(user_query, k=K_RESULTS)

  0%|          | 0/10 [00:00<?, ?it/s]

Calling LLM with sutta and user query...
Received response:  Does this sutta fully address the user query, why or why not: No, this sutta does not directly address the user's struggle with motivation to meditate. While it discusses the importance of mental purity and the effects of defilements on the mind, it does not provide specific guidance or practical advice for overcoming a lack of motivation to meditate. The focus is more on the overall understanding of mental clarity and its relation to good and bad outcomes rather than on developing a meditation practice.
[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a helpful librarian specializing in Buddhist texts and teachings.\nA user will provide a query, and you will be provided with a sutta that may or may not be \nrelevant to the query. Provide a brief statement with the following format:\n\nDoes this sutta fully address the user query, why or why not: {explanation} \n'}]}, {'role': 'user', 'content': [{'type': 'te

 10%|█         | 1/10 [00:03<00:33,  3.75s/it]

Calling LLM with sutta and user query...
Received response:  Does this sutta fully address the user query, why or why not: This sutta outlines a structured approach to meditation and gradual training in mindfulness, providing practical steps for progress in meditation practice. It illustrates how one can cultivate motivation through clear guidance and gradual development. However, it does not directly address the emotional or psychological barriers to finding motivation specifically, which could be pertinent to the user's struggle. Therefore, while it offers useful methodologies to motivate practice, it may not fully resonate with the user's specific struggles with motivation.
[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a helpful librarian specializing in Buddhist texts and teachings.\nA user will provide a query, and you will be provided with a sutta that may or may not be \nrelevant to the query. Provide a brief statement with the following format:\n\nDoes this 

 20%|██        | 2/10 [00:07<00:29,  3.67s/it]

...

Calling LLM with sutta and user query...
Received response:  Does this sutta fully address the user query, why or why not: No, this sutta primarily deals with the topic of caste and purity from a societal perspective, illustrating the Buddha’s teachings on the value of inner qualities over external status. It does not provide specific strategies or encouragement for cultivating motivation to meditate, which is the user's primary concern.
[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a helpful librarian specializing in Buddhist texts and teachings.\nA user will provide a query, and you will be provided with a sutta that may or may not be \nrelevant to the query. Provide a brief statement with the following format:\n\nDoes this sutta fully address the user query, why or why not: {explanation} \n'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': 'I struggle with motivation to meditate. What can I do?'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'tex

 90%|█████████ | 9/10 [00:33<00:03,  3.60s/it]

Calling LLM with sutta and user query...
Received response:  Does this sutta fully address the user query, why or why not: No, this sutta does not address the user’s struggle with motivation to meditate, as it focuses primarily on the dialogue between the Buddha and various deities, discussing concepts of knowledge, power, and the nature of existence, rather than providing practical guidance or encouragement related to personal meditation practice. It lacks specific advice or strategies for overcoming motivational barriers to meditation.
[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a helpful librarian specializing in Buddhist texts and teachings.\nA user will provide a query, and you will be provided with a sutta that may or may not be \nrelevant to the query. Provide a brief statement with the following format:\n\nDoes this sutta fully address the user query, why or why not: {explanation} \n'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': 'I struggle wi

100%|██████████| 10/10 [00:36<00:00,  3.67s/it]


TypeError: BaseModel.__init__() takes 1 positional argument but 3 were given

In [22]:

user_query = "I struggle with motivation to meditate. What can I do?"
results = scripture_search_tool.search(user_query, k=K_RESULTS)
scores = []
base_messages = [
    create_message("system", system_prompt),
    create_message("user", user_query),
]
for doc in tqdm(results):
    scores.append(query_for_score(base_messages, doc))
    sleep(1)
final_results = [Result(doc=doc, score=score) for doc, score in zip(results, scores)]

  0%|          | 0/10 [00:00<?, ?it/s]

Calling LLM with sutta and user query...
Received response:  Does this sutta fully address the user query, why or why not: No, this sutta does not directly address the user's struggle with motivation to meditate. While it discusses the importance of mindfulness and cultivating a pure mind free from defilements, it does not provide specific strategies or encouragement for enhancing motivation to engage in meditation practice. The focus is more on the qualities of mind and moral conduct rather than practical advice for overcoming barriers to meditation.
[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a helpful librarian specializing in Buddhist texts and teachings.\nA user will provide a query, and you will be provided with a sutta that may or may not be \nrelevant to the query. Provide a brief statement with the following format:\n\nDoes this sutta fully address the user query, why or why not: {explanation} \n'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': 

 10%|█         | 1/10 [00:03<00:31,  3.52s/it]

Calling LLM with sutta and user query...
Received response:  Does this sutta fully address the user query, why or why not: This sutta discusses a step-by-step training in meditation and mindfulness, providing practical guidance on how to develop the practice. It emphasizes the importance of virtuous living, guarding the senses, and moderation, which can help someone struggling with motivation to meditate. However, it may not directly address motivational techniques or personal encouragement tailored to individual struggles, making it partially relevant but not fully comprehensive in addressing the user's query about motivation.
[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a helpful librarian specializing in Buddhist texts and teachings.\nA user will provide a query, and you will be provided with a sutta that may or may not be \nrelevant to the query. Provide a brief statement with the following format:\n\nDoes this sutta fully address the user query, why or why not

 20%|██        | 2/10 [00:06<00:27,  3.38s/it]

...

 80%|████████  | 8/10 [00:30<00:07,  3.92s/it]

Calling LLM with sutta and user query...
Received response:  Does this sutta fully address the user query, why or why not: No, this sutta primarily discusses the caste system and the nature of purity and worthiness in relation to birth. It does not provide specific guidance on overcoming struggles with motivation to meditate or practical steps to enhance the meditation practice. For motivation in meditation, one might seek other teachings that focus on the benefits of meditation, overcoming obstacles, or developing mindfulness and commitment.
[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a helpful librarian specializing in Buddhist texts and teachings.\nA user will provide a query, and you will be provided with a sutta that may or may not be \nrelevant to the query. Provide a brief statement with the following format:\n\nDoes this sutta fully address the user query, why or why not: {explanation} \n'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': 'I strugg

 90%|█████████ | 9/10 [00:35<00:04,  4.13s/it]

Calling LLM with sutta and user query...
Received response:  Does this sutta fully address the user query, why or why not: No, this sutta does not directly address the user's struggle with motivation to meditate. The content is focused on the Buddha's confrontation with Baka Brahmā and Māra, illustrating the nature of awakening and the futility of attachment to worldly concepts. While it demonstrates the Buddha's supreme knowledge and powers, it does not provide practical guidance or encouragement for an individual looking for motivation to maintain a meditation practice.
[{'role': 'system', 'content': [{'type': 'text', 'text': 'You are a helpful librarian specializing in Buddhist texts and teachings.\nA user will provide a query, and you will be provided with a sutta that may or may not be \nrelevant to the query. Provide a brief statement with the following format:\n\nDoes this sutta fully address the user query, why or why not: {explanation} \n'}]}, {'role': 'user', 'content': [{'ty

100%|██████████| 10/10 [00:39<00:00,  3.94s/it]


In [25]:
final_results = sorted(final_results, key=lambda x: x.score, reverse=True)
final_results[0].score

8